<a href="https://colab.research.google.com/github/SupriyaRani/news_qa_cp/blob/develop/Elmo_Model_v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, Embedding
from tensorflow.keras.layers import SpatialDropout1D, Dense, Dropout, Input, concatenate, Conv1D, Activation, Flatten
from sklearn.preprocessing import LabelEncoder

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from tqdm import tqdm
from sklearn.svm import SVC
#from keras.models import Sequential
#from keras.layers.recurrent import LSTM, GRU
#from keras.layers.core import Dense, Activation, Dropout
#from keras.layers.embeddings import Embedding
#from keras.layers.normalization import BatchNormalization
#from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
#from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
#from keras.preprocessing import sequence, text
#from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
#stop_words = stopwords.words('english')
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, Embedding
from tensorflow.keras.layers import SpatialDropout1D, Dense, Dropout, Input, concatenate, Conv1D, Activation, Flatten
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm
import re
import time
import pickle
import os
pd.set_option('display.max_colwidth', 200)
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
elmo = hub.KerasLayer("https://tfhub.dev/google/elmo/2", signature='default')#.signatures["default"]
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv(r'/content/drive/MyDrive/sample_data.csv')

In [ ]:
full_data = pd.read_csv(r'/content/drive/MyDrive/combined-newsqa-data-v1.csv')

In [ ]:
full_data = full_data.drop_duplicates()

# New Section

In [ ]:
data.head()

,story_id,Question,paragraph,ans_st_index,ans_en_index,Question1
0,./cnn/stories/42d01e187213e86f5fe617fe32e716ff7fa3afc4.story,What was the amount of children murdered?,"NEW DELHI, India (CNN) -- A high court in northern India on Friday acquitted a wealthy businessman facing the death sentence for the killing of a teen in a case dubbed ""the house of horrors.""\n\n\...",294.0,297.0,what was the amount of children murdered
1,./cnn/stories/42d01e187213e86f5fe617fe32e716ff7fa3afc4.story,When was Pandher sentenced to death?,"NEW DELHI, India (CNN) -- A high court in northern India on Friday acquitted a wealthy businessman facing the death sentence for the killing of a teen in a case dubbed ""the house of horrors.""\n\n\...",261.0,271.0,when was pandher sentenced to death
2,./cnn/stories/42d01e187213e86f5fe617fe32e716ff7fa3afc4.story,The court aquitted Moninder Singh Pandher of what crime?,"NEW DELHI, India (CNN) -- A high court in northern India on Friday acquitted a wealthy businessman facing the death sentence for the killing of a teen in a case dubbed ""the house of horrors.""\n\n\...",624.0,640.0,the court aquitted moninder singh pandher of what crime
3,./cnn/stories/42d01e187213e86f5fe617fe32e716ff7fa3afc4.story,who was acquitted,"NEW DELHI, India (CNN) -- A high court in northern India on Friday acquitted a wealthy businessman facing the death sentence for the killing of a teen in a case dubbed ""the house of horrors.""\n\n\...",195.0,218.0,who was acquitted
4,./cnn/stories/42d01e187213e86f5fe617fe32e716ff7fa3afc4.story,who was sentenced,"NEW DELHI, India (CNN) -- A high court in northern India on Friday acquitted a wealthy businessman facing the death sentence for the killing of a teen in a case dubbed ""the house of horrors.""\n\n\...",195.0,218.0,who was sentenced


In [ ]:
print(data.shape)
data = data.dropna()
print(data.shape)

(201, 6)
(168, 6)


EDA

In [ ]:
data.isnull().sum()

story_id        0
Question        0
paragraph       0
ans_st_index    0
ans_en_index    0
Question1       0
dtype: int64

In [ ]:
for index,text in enumerate(data['paragraph'][35:40]):
  print('Review %d:\n'%(index+1),text)

In [ ]:
data['paragraph_cleaned']=data['paragraph'].apply(lambda x: x.lower())

In [ ]:
data['paragraph_cleaned']=data['paragraph_cleaned'].apply(lambda x: re.sub('\w*\d\w*','', x))

In [ ]:
# Importing spacy
import spacy

# Loading model
nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])

# Lemmatization with stopwords removal
data['lemmatized']=data['paragraph_cleaned'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))

In [ ]:
data_grouped=data[['Question','lemmatized']].groupby(by='Question').agg(lambda x:' '.join(x))
data_grouped.head()

,lemmatized
Question,
Census bureaus are hiring people from where?,cnn affiliates report job seeker find work country look employment cope situation . \n\n\n\n census employee pose new handheld device field worker use count . \n\n\n\n ( cnn ) -- nation roll f...
Egypt scored how many goals?,"( cnn ) -- goal minute start extra - time help defend champion egypt book place african nations cup semifinal battling - victory cameroon . \n\n\n\n\n\n game , , mar controversial goal midfielder ..."
Have the police found the man yet?,"berlin , germany ( cnn ) -- bomb scare affect flight munich airport germany wednesday false alarm , german police tell cnn . \n\n\n\n\n\n police launch huge manhunt wednesday passenger appear ..."
How long is the loan deal?,( cnn ) -- barcelona midfielder alexander hleb turn inter milan favor return bundesliga club stuttgart loan season . \n\n\n\n alexander hleb present medium complete loan club stuttgart . \n\n\n\n ...
How many Christian families have been displaced,"baghdad , iraq ( cnn ) -- , christians flee northern iraqi city mosul past week killing death threat , iraq ministry immigration displace person say thursday . \n\n\n\n christian family flee mos..."


In [ ]:
# Creating Document Term Matrix
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(analyzer='word')
data=cv.fit_transform(data_grouped['lemmatized'])
data_dtm = pd.DataFrame(data.toarray(), columns=cv.get_feature_names())
data_dtm.index=data_grouped.index
data_dtm.head(3)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,abbas,abc,abdel,abdulmutallab,ability,able,aboard,abu,academy,accept,access,acclaim,acclaimed,accompany,accomplish,accomplishment,accord,account,accurate,accuse,achieve,achievement,achille,acknowledge,acoustic,acqua,acquit,act,action,active,activity,actor,actual,actually,ad,add,addition,additional,address,adequate,...,willingness,win,wing,wish,witness,wobbling,wojtyla,wolff,woman,wonderful,woodson,wool,word,work,worker,world,wpri,wrangling,write,writer,wrong,wu,www,xinhua,yard,yeah,year,yellow,yes,yigal,yom,york,young,zac,zambia,zlatan,zoo,zuroff,zynga,лl
Question,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Census bureaus are hiring people from where?,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,4,8,0,1,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0
Egypt scored how many goals?,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,...,0,3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0
Have the police found the man yet?,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Transposing document term matrix
data_dtm=data_dtm.transpose()

In [ ]:
# Importing wordcloud for plotting word clouds and textwrap for wrapping longer text
from wordcloud import WordCloud
from textwrap import wrap
import matplotlib.pyplot as plt

# Function for generating word clouds
def generate_wordcloud(data,title):
  wc = WordCloud(width=400, height=330, max_words=250,colormap="Dark2").generate_from_frequencies(data)
  plt.figure(figsize=(10,8))
  plt.imshow(wc, interpolation='bilinear')
  plt.axis("off")
  plt.title('\n'.join(wrap(title,60)),fontsize=13)
  plt.show()
  
# Plotting word cloud for each product
for index,news in enumerate(data_dtm.columns):
  generate_wordcloud(data_dtm[news].sort_values(ascending=False),news)

In [ ]:
paragraph=data.paragraph
questions = data.Question1
ans_st_index = data.ans_st_index

In [ ]:
#elmo
elmo = hub.KerasLayer("https://tfhub.dev/google/elmo/2", trainable=True)

# just a random sentence
x = ["Roasted ants are a popular snack in Columbia"]

# Extract ELMo features 
#embeddings = elmo(x, signature="default", as_dict=True)["elmo"]
#embeddings.shape

In [ ]:
q_input_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="q_input_word_ids")

pooled_output, sequence_output = bert_layer([q_input_ids])
#clf_output = sequence_output[:, 0, :]
#net_q = tf.keras.layers.Dense(64, activation='relu')(clf_output)


In [ ]:
#elmo
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

# just a random sentence
x = ["Roasted ants are a popular snack in Columbia"]

# Extract ELMo features 
embeddings = elmo(x, signature="default", as_dict=True)["elmo"]
embeddings.shape

In [ ]:
def elmo_vectors(x):
    embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        #return average of ELMo features
        return sess.run(tf.reduce_mean(embeddings,1))
train = paragraph
list_train = [train[i:i+10] for i in range(0,train.shape[0],10)] #crashing for 100

# Extract ELMo embeddings
elmo_train = [elmo_vectors(x) for x in list_train]

elmo_train_new = np.concatenate(elmo_train, axis = 0)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
#save elmo_train_new
pickle_out = open("elmo_train_03032019.pickle","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

# load elmo_train_new
pickle_in = open("elmo_train_03032019.pickle", "rb")
elmo_train_new = pickle.load(pickle_in)

In [ ]:
elmo_train_new

In [ ]:
from sklearn.model_selection import train_test_split

xtrain, xvalid, ytrain, yvalid = train_test_split(elmo_train_new, 
                                                ans_st_index,  
                                                random_state=42, 
                                                train_size=0.8)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

lreg = LogisticRegression()
lreg.fit(xtrain, ytrain)

preds_valid = lreg.predict(xvalid)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
f1score = f1_score(yvalid, preds_valid, average ='micro')
print(f1score)
f1score = f1_score(yvalid, preds_valid, average ='weighted')
print(f1score)


0.14705882352941177
0.09509803921568627
